## Quality check of sample ID ##


### Background: ### 

A standardized sample naming schema is used to capture all associated meta-data prior to sequencing. Briefly, the `AMD ID` consist of 20 characters that capture information like collection year, geographical and treatment information, sample type and molecular markers included for each sample. 

Capturing this information at the pre-processing stage reduces the need to have multiple documents with this information, removing potential for mislabeling or tracking errors, and provides the bioinformatics team  with sufficient information to perform subsequent analysis and standardize analysis pipelines, including submission of data to NCBI. 

#### AMD ID Description #### 

* The AMD ID Key:  `<year> <country> <state/site> <day of failure> <treatment> <sample_id> <genus spp> <sample type> <mol marker bit code> <# sample processed>`. 
* Any missing meta data is replaced with an `x` _lower case_ strings for each character position. 

Example:
- `Individual` sequenced sample ID: `17GNDo00F0001PfF1290` = `<2017> <Guinea> <Dorota> <Day0> <AS+AQ> <0001> <P.falciparum> <FilterBloodSpot> <k13-crt-mdr-dhfr-dhps-cytB-cpmp-pfs47> <0>` 
                      
- `Pooled` sequenced sample ID: `17GNDo00x001p10F1290` = `<2017> <Guinea> <Dorota> <Day0> <missing info> <001> <Pooled Samples> <Samples in Pool> <FilterBloodSpot> <k13-crt-mdr-dhfr-dhps-cytB-cpmp-pfs47> <0>`
       
NOTE: If information is not availble (na), **x** is used for each character position. For example, in the pooled samples Treatment has (2) character spaces, represented as a two digit number. This is replaced with (2) **xx** since its a pool of samples that have possibly different treatment information. Moreover, for pooled samples, **Sample ID** is replaced with **three digit number + the letter p** (for pooled), and **Genus** is replaced with **total number of SamplesInPool** as a (2) digit number. 

Please see presentation at [AMD_ID presentation](https://github.com/CDCgov/MaRS/blob/master/Geneious_workflow/01_sample_ID_QC/files/AMD_ID_create_key.pptx) for more descriptive information. The `AMD ID` can be generated using the [AMD create template](https://github.com/CDCgov/MaRS/blob/master/Geneious_workflow/01_sample_ID_QC/files/AMD_ID_create_template.xlsx). 
     
### Code information ###

This code checks whether the `AMD ID` is the correct length and contains all the required elements described above. 
              

### Required packages
- [Pandas](https://pandas.pydata.org/) 
- [tabulate](https://pypi.org/project/tabulate/)

### Inputs
- Sample IDs in .csv file or raw fastq files

### Outputs:
- Outputs a list of sample IDs which does not match with standerdized AMD IDs.


In [ ]:
## Import dependencies ## 

import pandas as pd
import os
import re
import glob
import tabulate


## Please edit the path if it is csv or fastq folder ##

- If file is csv, add the full path with its file extension **EX: /User/admin/Desktop/sample.csv**

- If it is folder with paired fastq files, add the full path with its file extension **".fastq.gz or .fastq"** for all the fastq file **EX: "/User/admin/Desktop/Folder_with_fastq/*.fastq.gz (if it is gunzip)or /User/admin/Desktop/Folder_with_fastq/*.fastq"**




In [194]:
My_path = "/Users/dhruvipatel/Documents/Burkina_Faso/BF_Paired/*fastq.gz"             # Path is a folder containing a fastq files

#My_path = "/Users/dhruvipatel/Downloads/BF2021TES-PlatesP1-P2-10282022-SRMBEPLAK.csv"       # Path is Samplesheet csv file before Sequencing 

ext = os.path.splitext(My_path)[-1].lower()


if ext == ".csv":
        print (My_path, "is an csv!")
        My_CSV = pd.read_csv(My_path, header=None, skiprows = 20)                           # Skip thye first 20 rows from SampleSheet as they are not AMDID
        My_CSV.columns = My_CSV.iloc[0]     # Set 20th column as 
        My_CSV = My_CSV[1:].reset_index(drop=True)
   
        Sample_file = My_CSV.filter(['Sample_ID'], axis=1)                                  # Dataframe with only list of AMDIDs
        print("Total number of samples are:", len(Sample_file))

elif ext == ".gz" or ext == ".fastq":
        print (My_path, "is a fastq  file!")
        my_file = [f for f in glob.glob(My_path)]                                           # use glob functio to list the files
            
        clean_filenames = [doc_name.split("/")[-1].split("_")[0] for doc_name in my_file]                             
        Sample_file = pd.DataFrame(clean_filenames, columns=["Sample_ID"])                     # add column name to data frame called AMD_ID
        Sample_file = Sample_file.drop_duplicates()                                         # drop duplicates from list
        print("Total number of samples are:", len(Sample_file))
else:
        print (My_path, "is an unknown file format.")
        


/Users/dhruvipatel/Documents/Burkina_Faso/BF_Paired/*fastq.gz is a fastq  file!
Total number of samples are: 790


In [195]:
# This code will count the samples by sites for individual and Pooled. 
from collections import Counter

LT1 = Sample_file['Sample_ID'].tolist()             # convert column to list

IND = []
Pool = []

for i in LT1:                                       # separate pools and individual in separate list
    if i[12:13]== 'p':
        Pool.append(i)
        
    else:
        IND.append(i)
                           
LT_IND = []                
ind = [LT_IND.append(i[0:6]) for i in IND]          # count the number of individual samples per site

print ("The Individual samples count are:", dict(Counter(LT_IND)))

LT_Pool = [] 
pool = [LT_Pool.append(j[0:6]) for j in Pool]       # count the number of Pools samples per site


print ("\nThe Pooled samples count are: ", dict(Counter(LT_Pool)))   

The Individual samples count are: {'21BFGO': 206, '21BFNI': 196, '21BFNA': 236}

The Pooled samples count are:  {'21BFGO': 52, '21BFNI': 53, '21BFNA': 47}


In [ ]:
## Creat a empty list for AMD_IDs 

Sample_no_match = []        # All the Ids with no match will be saved in list
Sample_with_match = []      # all the ID which has length  20 will be saved in list
 
## First part is to check if Sample ID has length 20 or not

Sample_name = Sample_file.rename(columns={'Sample':"Sample_ID", 'AMD_ID': "Sample_ID",'AMD ID (Pooled)': "Sample_ID", 'Document Name': "Sample_ID"})      # rename column name to Sample_ID as differant files migth have diffenrt column name.
  
SampleID_df = Sample_name[['Sample_ID']]                       # creat a dataframe using the column Sample_ID 

#remove US conrtols to avoid any errors in sample ID

SampleID_df = SampleID_df[SampleID_df['Sample_ID'].str.contains("USxx|USGA|NTC|3D7|HB3|7G8|DD2|D6") == False]


for rows in SampleID_df.index:                                 # run a for loop on each rows
    
    sample_name =SampleID_df['Sample_ID'][rows].split('/n')    # split rows by newline
    for each_ID in sample_name:
        if len(each_ID) == 20 :                                # if length is 20, save the samples in Sample_with_match list
            Sample_with_match.append(each_ID) 
        else: 
            Sample_no_match.append(each_ID)                    # if length is not 20 then save the results in Sample_no_match list. 
            print(each_ID,"has length", int(len(each_ID)))     # print the sample ID with its length if less than 20



In [ ]:
## 2nd part is to check all ID with length 20, if it matches with AMD ID information regular expression as shown in discription at begining.

for each_file  in Sample_with_match:                                     # Run a for loop for each file in Sample_with_match list
    
    AMD_ID =('([0-9]{2})([A-Zx]{2})([A-Za-z]{2})([0-9x]{2})([A-Zx]{1})([0-9]{3})(([0-9]{1})|([p]{1}))(([0-9]{2})|([Pf]{2}))([A-Zx]{1})([0-9x]{3})([0-9]{1})')
             
                                                                         # split AMD ID by its information using regular expression
   
    AMD_group = re.match(AMD_ID,each_file)                               # match each ID with pattern
    
    if AMD_group is None :                                               # if match does not found
            
        Sample_no_match.append(each_file)                                # append the ID to list
        print(each_file, "is not maching with ID")
        
    else:
       
        pass                                                             # if ID match with regex, pass
#print(Sample_with_match)



In [ ]:
## lastly, print All the IDs without match so that user can review them and make a corrction before further processing.
print(len(Sample_no_match), "out of", len(SampleID_df),"samples did not match with AMD_ID")         # print the total number of samples that did not match 


if len(Sample_no_match) == 0:
        print("you are good to proceed with analysis: All the samples pass through QC test")
else :
    print("\nHere is the list of samples that did not match")
    
ID_No_match = "\n".join ([str(ID) for ID in Sample_no_match if len(Sample_no_match) != 0 ])         #  print the list of IDs that did not match 
print(ID_No_match)


In [ ]:
# This part of code runs through the samples_no_match list and creats a table with key. Then user can identyfy where the key does not match visually from the table.

data_regex_QC = []                               

#Sample_no_match = ["17GNDo00F0001PfF1291", "17GNDo00F0001PfF129","17GNDo00F0001PfF12911",'17GNDo00F0001PfF1','17GNDo0F0001PfF1291', "NF54","NTC-DFR", "NTC-DHFR" ]

# Loop through the Sample id with no match list, split the ID by key using regex and creat dictionary .

for id in Sample_no_match:
    if len(id) >= 15:
        match = re.match(r"(?P<year>\w{2})(?P<country>\w{2})(?P<Site>\w{2})(?P<Treatment_Day>\w{2})(?P<Treatment>\w{1})(?P<ID>\w{4})(?P<Genus_Pooled>\w{,2})(?P<Type>\w{,1})(?P<GenemarkerCode>\w{,3})(?P<Repeat>\w{0,})", id)
        dic = match.groupdict()
        Dict_QC_re ={"name": id,"length_of_sample_ID" : len(id)}              # append the two keys to dict for Sample name and its length
        Dict_QC_re.update(dic)                                                # update a dict with new key value pair i.e name and length
        data_regex_QC.append(Dict_QC_re)     
    elif len(id) < 15:
        Dict_QC_re ={"name": id,"length_of_sample_ID" : len(id)}              # append the two keys to dict for Sample name and its length
        data_regex_QC.append(Dict_QC_re)     
                                   
        
if len(data_regex_QC) != 0:                                            # If length of list is not 0; 
    header = data_regex_QC[0].keys()                                   # header = keys of dict
    rows = [x.values() for x in data_regex_QC]                         # rows will be value of dict
    print (tabulate.tabulate(rows, header, tablefmt="grid"))           # use tabulate module to creat a table   

else:
    print('\n',"All the samples are matching with AMD_ID","\n", "No errors found in samples")        # If all the IDs matched with AMD id no table will be created. 
     